# imports 

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 

In [4]:
# from ctgan import CTGAN
# from ctgan import load_demo

In [5]:
from sklearn import model_selection # split함수이용
from sklearn import ensemble # RF,GBM
from sklearn import metrics 

In [6]:
def down_sample_textbook(df):
    df_majority = df[df.is_fraud==0].copy()
    df_minority = df[df.is_fraud==1].copy()
    df_maj_dowsampled = sklearn.utils.resample(df_majority, n_samples=len(df_minority), replace=False, random_state=42)
    df_downsampled = pd.concat([df_minority, df_maj_dowsampled])
    return df_downsampled

# Overview

## 데이터 종류 

- fraudTrain.csv: (1048575, 23), 기본데이터
- df02: (214520, 23), is_fraud==0 에서는 20퍼의 샘플만, is_fraud==1 에서는 모든 샘플을 뽑아서 정리한 새로운 자료
- df50 = (12012, 23), df20에서 is_fraud==0 와 is_fraud==1 의 비율을 맞추어서 샘플을 뽑은 것 

|데이터|shape|사기거래빈도|설명|
|-|-|-|-|
|fraudTrain|(1048575, 22)|0.00573|원래자료|
|df02|(214520, 22)|0.028| is_fraud==0 에서는 20퍼의 샘플만, is_fraud==1 에서는 모든 샘플을 뽑아서 정리한 새로운 자료|
|df50|(12012, 22)|0.5| df02에서 사기비율을 50퍼로 맞추어 샘플링한 자료|
|df50_tr|(9009, 22)|0.49828| df50에서 랜덤으로 train/test를 분리하여 얻은 train dataset|
|df50_test|(3003, 22)|0.50516| df50에서 랜덤으로 train/test를 분리하여 얻은 test dataset|
|df02_tr|(211517, 22)|0.02122| df02에서 df50_test에 해당하는 인덱스를 제외|
|fraudTrain_tr|(1045572, 22)|0.00429| fraudTrain에서 df50_test에 해당하는 인덱스를 제외|

`-` fraudTrain

In [7]:
fraudTrain = pd.read_csv("fraudTrain.csv").iloc[:,1:]

In [8]:
fraudTrain.columns

Index(['trans_date_trans_time', 'cc_num', 'merchant', 'category', 'amt',
       'first', 'last', 'gender', 'street', 'city', 'state', 'zip', 'lat',
       'long', 'city_pop', 'job', 'dob', 'trans_num', 'unix_time', 'merch_lat',
       'merch_long', 'is_fraud'],
      dtype='object')

In [9]:
fraudTrain['is_fraud']

0          0
1          0
2          0
3          0
4          0
          ..
1048570    0
1048571    0
1048572    0
1048573    0
1048574    0
Name: is_fraud, Length: 1048575, dtype: int64

In [10]:
fraudTrain.is_fraud.mean().round(5)

0.00573

`-` df20

In [11]:
_df1 = fraudTrain[fraudTrain["is_fraud"] == 0].sample(frac=0.20, random_state=42)
_df2 = fraudTrain[fraudTrain["is_fraud"] == 1]
df02 = pd.concat([_df1,_df2])
df02.shape

(214520, 22)

In [12]:
df02.is_fraud.mean().round(5)

0.028

`-` df50

In [13]:
df50 = down_sample_textbook(df02)
df50.shape

NameError: name 'sklearn' is not defined

In [14]:
df50

NameError: name 'df50' is not defined

In [17]:
df50.is_fraud.mean().round(5)

0.5

`-` df50_tr, df50_test 

In [18]:
df50_tr,df50_test = sklearn.model_selection.train_test_split(df50, random_state=42)

In [19]:
df50_tr.is_fraud.mean().round(5), df50_test.is_fraud.mean().round(5)

(0.49828, 0.50516)

`-` df02_tr, fraudTrain_tr

In [20]:
df02_tr = df02.loc[[i not in df50_test.index for i in df02.index],:].copy()
fraudTrain_tr = fraudTrain.loc[[i not in df50_test.index for i in fraudTrain.index],:].copy()

In [21]:
df02_tr.shape, fraudTrain_tr.shape

((211517, 22), (1045572, 22))

In [22]:
df02_tr.is_fraud.mean().round(5), fraudTrain_tr.is_fraud.mean().round(5)

(0.02122, 0.00429)

# 분석방법정리 

||Train|Test|모형|설명변수|참고|
|-|-|-|-|-|-|
|분석1|df02_tr|df50_test|그레디언트부스팅|amt|base|
|**분석2**|df02_tr|df50_test|그레디언트부스팅|amt|가장좋음|

# 분석1

`-` step1: data 

In [26]:
X = np.array(df02_tr.loc[:,['amt']])
XX = np.array(df50_test.loc[:,['amt']])
y = np.array(df02_tr.is_fraud)
yy = np.array(df50_test.is_fraud)

`-` step2: lrnr 생성 

In [27]:
lrnr = ensemble.GradientBoostingClassifier()

`-` step3: fit 

In [28]:
lrnr.fit(X,y)

GradientBoostingClassifier()

`-` step4: evaluate

In [29]:
thresh = y.mean()
yyhat = (lrnr.predict_proba(XX)> thresh)[:,-1]
#yyhat = lrnr.predict(XX) 

In [30]:
metrics = [sklearn.metrics.accuracy_score,
           sklearn.metrics.precision_score,
           sklearn.metrics.recall_score,
           sklearn.metrics.f1_score]

In [31]:
_results2= pd.DataFrame({m.__name__:[m(yy,yyhat).round(6)] for m in metrics},index=['분석2'])
_results2

,accuracy_score,precision_score,recall_score,f1_score
분석2,0.88678,0.910677,0.86025,0.884746


# 분석2

`-` step1: data 

In [ ]:
# Names of the columns that are discrete
discrete_columns = ['trans_date_trans_time', 'cc_num', 'merchant', 'category', 'first', 'last', 'gender', 'street', 'city', 'state', 'zip', 'job', 'dob', 'trans_num', 'is_fraud']
ctgan = CTGAN(epochs=10)
ctgan.fit(df, discrete_columns)

# Create synthetic data
df2 = ctgan.sample(1000)

In [37]:
df02_tr.columns

Index(['trans_date_trans_time', 'cc_num', 'merchant', 'category', 'amt',
       'first', 'last', 'gender', 'street', 'city', 'state', 'zip', 'lat',
       'long', 'city_pop', 'job', 'dob', 'trans_num', 'unix_time', 'merch_lat',
       'merch_long', 'is_fraud'],
      dtype='object')

In [ ]:
['trans_date_trans_time', 'cc_num', 'merchant', 'category', 'first', 'last', 'gender', 'street', 'city', 'state', 'zip', 'job', 'dob', 'trans_num', 'is_fraud']

In [36]:
df02_tr

,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
669418,2019-10-12 18:21,4.089100e+18,"fraud_Haley, Jewess and Bechtelar",shopping_pos,7.53,Debra,Stark,F,686 Linda Rest,Kilgore,...,32.3836,-94.8653,24536,Multimedia programmer,1983-10-14,d313353fa30233e5fab5468e852d22fc,1350066071,32.202008,-94.371865,0
32567,2019-01-20 13:06,4.247920e+12,fraud_Turner LLC,travel,3.79,Judith,Moss,F,46297 Benjamin Plains Suite 703,Washington Court House,...,39.5370,-83.4550,22305,Television floor manager,1939-03-09,88c65b4e1585934d578511e627fe3589,1327064760,39.156673,-82.930503,0
156587,2019-03-24 18:09,4.026220e+12,fraud_Klein Group,entertainment,59.07,Debbie,Payne,F,204 Ashley Neck Apt. 169,Preston,...,41.5224,-71.9934,4720,Broadcast presenter,1977-05-18,3bd9ede04b5c093143d5e5292940b670,1332612553,41.657152,-72.595751,0
1020243,2020-02-25 15:12,4.957920e+12,fraud_Monahan-Morar,personal_care,25.58,Alan,Parsons,M,0547 Russell Ford Suite 574,Kirk,...,39.6171,-102.4776,207,Network engineer,1955-12-04,19e16ee7a01d229e750359098365e321,1361805120,39.080346,-103.213452,0
116272,2019-03-06 23:19,4.178100e+15,fraud_Kozey-Kuhlman,personal_care,84.96,Jill,Flores,F,639 Cruz Islands,Baroda,...,41.9488,-86.4913,3104,"Horticulturist, commercial",1981-03-29,a0c8641ca1f5d6e243ed5a2246e66176,1331075954,42.502065,-86.732664,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1047054,2020-03-10 3:40,3.546670e+15,fraud_Kunze Inc,grocery_pos,289.37,Jordan,May,M,1626 Susan Course,Aledo,...,32.7004,-97.6039,13602,Optometrist,1984-07-05,70b8c60c7bc2ed6e82505c8018919b1f,1362886857,32.238135,-98.129466,1
1047089,2020-03-10 3:59,3.589290e+15,fraud_Kris-Weimann,misc_net,690.49,Paula,Estrada,F,350 Stacy Glens,Spencer,...,43.7557,-97.5936,343,"Development worker, international aid",1972-03-05,fb1ddd251bbec9b84c9755e856d51723,1362887989,43.254214,-98.267759,1
1047157,2020-03-10 4:31,3.546670e+15,"fraud_Casper, Hand and Zulauf",grocery_pos,324.74,Jordan,May,M,1626 Susan Course,Aledo,...,32.7004,-97.6039,13602,Optometrist,1984-07-05,4dca0549e43b7e265cae7fd8a7e563b4,1362889904,33.607221,-97.996506,1
1047208,2020-03-10 4:59,3.589290e+15,fraud_Kiehn Inc,grocery_pos,331.33,Paula,Estrada,F,350 Stacy Glens,Spencer,...,43.7557,-97.5936,343,"Development worker, international aid",1972-03-05,d18c55035998e461aa9040e254b74925,1362891561,44.228731,-98.330520,1


In [26]:
X = np.array(df02_tr.loc[:,['amt']])
XX = np.array(df50_test.loc[:,['amt']])
y = np.array(df02_tr.is_fraud)
yy = np.array(df50_test.is_fraud)

`-` step2: lrnr 생성 

In [27]:
lrnr = ensemble.GradientBoostingClassifier()

`-` step3: fit 

In [28]:
lrnr.fit(X,y)

GradientBoostingClassifier()

`-` step4: evaluate

In [29]:
thresh = y.mean()
yyhat = (lrnr.predict_proba(XX)> thresh)[:,-1]
#yyhat = lrnr.predict(XX) 

In [30]:
metrics = [sklearn.metrics.accuracy_score,
           sklearn.metrics.precision_score,
           sklearn.metrics.recall_score,
           sklearn.metrics.f1_score]

In [31]:
_results2= pd.DataFrame({m.__name__:[m(yy,yyhat).round(6)] for m in metrics},index=['분석2'])
_results2

,accuracy_score,precision_score,recall_score,f1_score
분석2,0.88678,0.910677,0.86025,0.884746
